Check that use of `queryDatasets` for overlapping images is returning what we want.

Import packages and set up the butler.

In [ ]:
import matplotlib.pyplot as plt
import lsst.daf.butler as dafButler

In [ ]:
butler = dafButler.Butler('dp02', collections='2.2i/runs/DP0.2')

In [ ]:
registry = butler.registry

Define funtion for getting corners as arrays of RA and Dec.

In [ ]:
def get_corners_radec(wcs, bbox):

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    
    corners_ra = []
    corners_dec = []
    
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

Use case.

The visit and detector are known, and the dataId is defined.

In [ ]:
dataId = {'visit': 192350, 'detector': 175}
visitInfo = butler.get('calexp.visitInfo', dataId=dataId)
print(visitInfo)

Get corners.

In [ ]:
calexp_wcs = butler.get('calexp.wcs', dataId=dataId)
calexp_bbox = butler.get('calexp.bbox', dataId=dataId)
calexp_corners_ra, calexp_corners_dec = get_corners_radec(calexp_wcs, calexp_bbox)
for i in range(4):
    print(calexp_corners_ra[i], calexp_corners_dec[i])

Get the list of overlapping deepCoadd tract and patches with `queryDatasets`.

In [ ]:
for ref in registry.queryDatasets("deepCoadd", dataId=dataId):
    print(ref)

Do all these patches really overlap with the single visit and detector?

Plot all the bboxes.

In [ ]:
fig = plt.figure()

xvals = []
yvals = []
for i in range(4):
    xvals.append(calexp_corners_ra[i])
    yvals.append(calexp_corners_dec[i])
xvals.append(calexp_corners_ra[0])
yvals.append(calexp_corners_dec[0])
plt.plot(xvals, yvals, ls='solid', color='blue', label='visit detector')
del xvals, yvals

for r, ref in enumerate(registry.queryDatasets("deepCoadd", dataId=dataId)):
    deepCoadd_dataId = ref.dataId
    deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=deepCoadd_dataId)
    deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=deepCoadd_dataId)
    deepCoadd_corners_ra, deepCoadd_corners_dec = get_corners_radec(deepCoadd_wcs, deepCoadd_bbox)

    xvals = []
    yvals = []
    for i in range(4):
        xvals.append(deepCoadd_corners_ra[i])
        yvals.append(deepCoadd_corners_dec[i])
    xvals.append(deepCoadd_corners_ra[0])
    yvals.append(deepCoadd_corners_dec[0])
    plt.plot(xvals, yvals, ls='solid', lw=1, color='grey')
    del xvals, yvals
    del deepCoadd_dataId, deepCoadd_wcs, deepCoadd_bbox
    del deepCoadd_corners_ra, deepCoadd_corners_dec

plt.xlabel('RA')
plt.ylabel('Dec')
plt.legend()
plt.show()